In [1]:
registers = []

with open('inputs/day17.txt') as input:
    for line in input.readlines():
        if line.startswith('Register'):
            registers.append(int(line.strip().split(': ')[1]))
        elif line.startswith('Program'):
            program = [int(x) for x in line.strip().split(': ')[1].split(',')]

registers, program

([47719761, 0, 0], [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6, 5, 5, 3, 0])

In [2]:
def get_combo_operand(registers, ope):
    match ope:
        case 0 | 1 | 2 | 3:
            return ope
        case 4 | 5 | 6:
            return registers[ope - 4]
    raise AssertionError()

def run(program, registers):
    program = program[:]
    registers = registers[:]
    ins_ptr = 0
    output = []
    while ins_ptr < len(program):
        ins = program[ins_ptr]
        literal = program[ins_ptr + 1]
        combo = get_combo_operand(registers, program[ins_ptr + 1])
        match ins:
            case 0:
                # adv
                num = registers[0]
                den = 2**combo
                registers[0] = num // den
            case 1:
                # bxl
                registers[1] ^= literal
            case 2:
                # bst
                registers[1] = combo % 8
            case 3:
                # jnz
                if registers[0]:
                    ins_ptr = literal - 2
            case 4:
                # bxc
                registers[1] ^= registers[2]
            case 5:
                # out
                output.append(combo % 8)
            case 6:
                # bdv
                num = registers[0]
                den = 2**combo
                registers[1] = num // den
            case 7:
                # cdv
                num = registers[0]
                den = 2**combo
                registers[2] = num // den
        ins_ptr += 2
    return output


print(','.join(str(x) for x in run(program, registers)))

7,0,3,1,2,6,3,7,1


In [3]:
print(program)

[2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6, 5, 5, 3, 0]


In [4]:
def combo_string(arg):
    match arg:
        case 0 | 1 | 2 | 3: return str(arg)
        case 4: return 'A'
        case 5: return 'B'
        case 6: return 'C'
    
def print_program(program):
    for idx, (insn, arg) in enumerate(zip(program[::2], program[1::2])):
        print(idx, end=': ')
        match insn:
            case 0:
                print(f'A //= 2**{combo_string(arg)}')
            case 1:
                print(f'B ^= {arg}')
            case 2:
                print(f'B = {combo_string(arg)} % 8')
            case 3:
                print(f'if A != 0: jump {arg//2}')
            case 4:
                print(f'B ^= C')
            case 5:
                print(f'print({combo_string(arg)} % 8)')
            case 6:
                print(f'B = A // 2**{combo_string(arg)}')
            case 7:
                print(f'C = A // 2**{combo_string(arg)}')

print_program(program)

0: B = A % 8
1: B ^= 5
2: C = A // 2**B
3: A //= 2**3
4: B ^= C
5: B ^= 6
6: print(B % 8)
7: if A != 0: jump 0


Simplifying/converting to Python:
```python
while True:
    B = (A % 8) ^ 5
    C = A // 2**B
    A //= 8
    B ^= C ^ 6
    print(B % 8)
    if A == 0: break
```
Simplifying further:
```python
while True:
    print(((A % 8) ^ 3 ^ (A // 2**((A % 8) ^ 5))) % 8)
    A //= 8
    if A == 0: break
```
This gives a set of 16 equations,
$$ (((A >> 3k) \mod 8) \oplus 3 \oplus ((A >> 3k) >> ((A >> 3k) \mod 8) \oplus 5)) \mod 8 = r_k $$
for $k = 0, \ldots, 15$, and $r_k = \texttt{program[k]}$.

Unclear how to solve this directly because of the variable bit-shift operation so resort to search.

In [5]:
# A will have up to 48 bits so that it outputs 16 numbers, plus we need 8 more zero bits to account for the bit-shift operation
bits = [None] * 56
smallest_A = None

def get_triple(base):
    vals = bits[base:base+3]
    if any(b is None for b in vals): return None
    return int(''.join(str(x) for x in vals[::-1]), base=2)

def set_triple(base, val):
    for i in range(3):
        bval = (val >> i) & 1
        if bits[base + i] is not None and bits[base + i] != bval:
            # already set to something different
            return False
    for i in range(3):
        if bits[base + i] is None:
            bits[base + i] = (val >> i) & 1
    return True

def unset_triple(base, old):
    for i in range(3):
        bits[base+i] = old[i]


def build_A():
    A = 0
    for i, b in enumerate(bits):
        bval = 0 if b is None else b
        A |= (bval << i)
    return A


def search(k):
    """
    The idea: have 16 constraints, one for each k.
    48 bits of A - 15 triples.
    If triple tk unknown, try values 0...7.
    Once chosen, compute tk ^ 5.
    offset = 3*k+sk
    uk = program[k]^tk^3
    Then enfore constraint (A >> offset) & 7 = uk
    If this conflicts, backtrack.
    """
    global smallest_A
    if k == 16:
        res = build_A()
        if smallest_A is None or res < smallest_A:
            smallest_A = res
        return
    tk = 3*k
    tkval = get_triple(tk)
    if tkval is not None:
        # triple is known
        sk = tkval ^ 5
        offset = 3*k + sk
        assert offset+2 < len(bits)
        uk = program[k] ^ tkval ^ 3
        current = get_triple(offset)
        if current is None:
            oldu = bits[offset:offset+3]
            if set_triple(offset, uk):
                search(k + 1)
            unset_triple(offset, oldu)
        else:
            if current == uk:
                search(k+1)
    else:
        # triple is unknown: try all 8 possible values
        old = bits[tk:tk+3]
        for val in range(8):
            if not set_triple(tk, val):
                unset_triple(tk, old)
                continue
            sk = val ^ 5
            offset = 3*k + sk
            assert offset+2 < len(bits)
            uk = program[k] ^ val ^ 3
            oldu = bits[offset:offset+3]
            currentu = get_triple(offset)
            if currentu is None:
                if set_triple(offset, uk):
                    search(k+1)
                unset_triple(offset, oldu)
            else:
                if currentu == uk:
                    search(k+1)
            unset_triple(tk, old)

search(0)
smallest_A

109020013201563